# Data Preprocessing 

In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from numpy import random
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score

from scipy.stats import skew, norm
from feature_engine.transformation import YeoJohnsonTransformer
from category_encoders.binary import BinaryEncoder

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import xgboost as xgb
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from mlxtend.regressor import StackingRegressor
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.kernel_ridge import KernelRidge
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from mlxtend.regressor import StackingCVRegressor

In [3]:
from sklearn.feature_selection import mutual_info_regression
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [4]:
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')
train = pd.read_csv('train.csv')

In [5]:
train.shape, test.shape

((1460, 81), (1459, 80))

Based on an initial analysis the train and test datasets have similar characteristics, so it will be easier to combine them for the data preprocessing work. 

In [6]:
train_test = pd.concat([train, test], ignore_index=True)

In [7]:
train_test.shape

(2919, 81)

To show the amount of 'NaN' values in each column of the dataset. 

In [8]:
pd.isnull(train_test).sum()[pd.isnull(train_test).sum() > 0]

MSZoning           4
LotFrontage      486
Alley           2721
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        24
MasVnrArea        23
BsmtQual          81
BsmtCond          82
BsmtExposure      82
BsmtFinType1      79
BsmtFinSF1         1
BsmtFinType2      80
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
Electrical         1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu     1420
GarageType       157
GarageYrBlt      159
GarageFinish     159
GarageCars         1
GarageArea         1
GarageQual       159
GarageCond       159
PoolQC          2909
Fence           2348
MiscFeature     2814
SaleType           1
SalePrice       1459
dtype: int64

It looks like Alley, FireplaceQu, PoolQC, Fence and MiscFeature have significant numbers of missing data. At first those columns were eliminated. However, the results were better if these columns were retained and the missing data were coded as 'None', assuming these features didn't exist for their related houses. 

There are a number of rows that have less than 5 rows with missing data. Since some of these are categorical and some are continuous data, their missing data will be replaced with the most frequent value. 

In [9]:
drop_high_nan=['Alley','FireplaceQu','PoolQC','Fence','MiscFeature']
train_test[drop_high_nan] = train_test[drop_high_nan].fillna('None')

small_nan_cols = ['MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 
                  'TotalBsmtSF', 'Electrical', 'BsmtFullBath', 'BsmtHalfBath', 'KitchenQual', 'Functional', 'GarageCars', 
                  'GarageArea','SaleType', 'SaleCondition']
small_impute = SimpleImputer(strategy='most_frequent')
train_test[small_nan_cols] = pd.DataFrame(small_impute.fit_transform(train_test[small_nan_cols]),columns=small_nan_cols)

The following columns seem to have one value significantly larger than the rest, and it would probably be best to use the mode, or most common, value to feel each NaN value: MasVnrType, MasVnrArea, BsmtCond, BsmtExposure, BsmtFinType2, GarageType, GarageFinish, GarageQual, and GarageCond. That represents 9 out of the 13 columns. 

BsmtQual has two values larger than the rest: Gd and TA. But it only has 2.8% NaNs, so simply using the mode might be good enough.  

GarageYrBlt has 59 NaNs out 2919 rows which is only 2%. It has a dispersed set of values, so it might be easiest just to have any NaNs have the same value as YearBuilt. 

BsmtFinType1 has only 2.7% value of NaNs, and most two of its largest values are GLQ and Unf. It might be easiest to use the mode here. 

LotFrontage has 486 NaNs out of 2919 rows which is a pretty high 16.7%. It has a dispersed range of values, but looking at its characteristics from the describe function above, it seems to have a pretty even distribution with a mean of 10,168 and a median of 9,453. So using the mean to fill in the NaNs seems reasonable.

In [10]:
mode_cols = ['MasVnrType', 'MasVnrArea', 'BsmtCond', 'BsmtExposure', 'BsmtFinType2', 'BsmtQual', 'BsmtFinType1']
mode_impute = SimpleImputer(strategy='most_frequent')
train_test[mode_cols] = pd.DataFrame(mode_impute.fit_transform(train_test[mode_cols]),columns=mode_cols)

garage_cols = ['GarageType', 'GarageFinish', 'GarageQual','GarageCond', 'GarageYrBlt']
train_test[garage_cols] = train_test[garage_cols].fillna('None')

train_test['LotFrontage'].fillna((train_test['LotFrontage'].mean()), inplace=True)

A final check to ensure that the only missing values are the expected SalePrice data from the test dataset. 

In [11]:
pd.isnull(train_test).sum()[pd.isnull(train_test).sum() > 0]

SalePrice    1459
dtype: int64

The BsmtQual and BsmtFinType1 have ordinal scaled categorical values with an inherent order. So the conversion from categorical to numerical was done manually. 

In [12]:
train_test['BsmtQual'].unique()

array(['Gd', 'TA', 'Ex', 'Fa'], dtype=object)

In [13]:
train_test['BsmtFinType1'].unique()

array(['GLQ', 'ALQ', 'Unf', 'Rec', 'BLQ', 'LwQ'], dtype=object)

In [14]:
train_test.BsmtQual = train_test.BsmtQual.replace({"Ex": 110, "Gd": 95, "TA": 85, "Fa": 75, "Po": 60, "NA": 0})
train_test.BsmtFinType1 = train_test.BsmtFinType1.replace({"GLQ": 6, "ALQ": 5, "BLQ": 4, "Rec": 3, "LwQ": 2, "Unf": 1,
                                                         "NA": 0})

To look at how many features have a skew above 0.6, since high skew can be an issue in regression analysis. For some reason, the results were better with a skew limit of 0.6 than the more standard value of 0.5. Skewness measures how symmetrical a distribution of data is. Data with a skew of 0 is perfectly symmetrical.

In [15]:
number_cols = train_test.select_dtypes(include=np.number).columns.tolist()

In [16]:
skew_features = train_test[number_cols].apply(lambda x: skew(x)).sort_values(ascending=False)
high_skew = skew_features[skew_features > 0.6]
skew_index = high_skew.index
skew_index

Index(['MiscVal', 'PoolArea', 'LotArea', 'LowQualFinSF', '3SsnPorch',
       'KitchenAbvGr', 'EnclosedPorch', 'ScreenPorch', 'OpenPorchSF',
       'WoodDeckSF', 'LotFrontage', '1stFlrSF', 'MSSubClass', 'GrLivArea',
       '2ndFlrSF', 'BsmtQual', 'TotRmsAbvGrd', 'Fireplaces', 'HalfBath'],
      dtype='object')

In [17]:
print("There are {} numerical features with Skew > 0.6 :".format(high_skew.shape[0]))
skewness = pd.DataFrame({'Skew' :high_skew})
skewness

There are 19 numerical features with Skew > 0.6 :


,Skew
MiscVal,21.947195
PoolArea,16.898328
LotArea,12.822431
LowQualFinSF,12.088761
3SsnPorch,11.376065
KitchenAbvGr,4.302254
EnclosedPorch,4.003891
ScreenPorch,3.946694
OpenPorchSF,2.535114
WoodDeckSF,1.842433


To normalize the features with skew above 0.6 the Yeo-Johnson Transformation is used since data with negative values or values of zero can be included. This transformation is a way to transform a continuous variable so that the output looks more normally distributed. 

In [18]:
yjt = YeoJohnsonTransformer()
yjt.fit(train_test[skew_index])
train_test[skew_index] = yjt.transform(train_test[skew_index])

Since most models cannot work with categorical data, it will be necessary to identify all the columns that have non-numeric object values and then convert them to numeric values. The best approach for this conversion seems to be the straightforward python factorize function. 

In [19]:
obj_cols = list(train_test.select_dtypes(['object']).columns)

In [20]:
for column in obj_cols:
     train_test[column] = pd.factorize(train_test[column], sort=True)[0]

To create some new columns that might compound the effects of some of the existing columns which have higher impacts the score. eli5 (https://eli5.readthedocs.io/en/latest/overview.html) was used with an xgbregressor model with its default parameters on an earlier workbook version to measure the impact of each column on the overall score. 

In [21]:
train_test['QualCondSum'] = train_test['OverallQual'] + train_test['OverallCond']
train_test['RemodTime'] = train_test['YearRemodAdd'] - train_test['YearBuilt']
train_test['BsmtFinTypeSF1'] = train_test['BsmtFinType1'] * train_test['BsmtFinSF1']
train_test['TotalFlrSF'] = train_test['1stFlrSF'] + train_test['2ndFlrSF']
train_test['TotalFinSF'] = train_test['GrLivArea'] + train_test['BsmtFinSF1']
train_test['GarageCarArea'] = train_test['GarageArea'] * train_test['GarageCars']
train_test['TotalSF'] = train_test['1stFlrSF'] + train_test['2ndFlrSF'] + train_test['TotalBsmtSF']

# Additional Feature Engineering

After completing this model, I took the Kaggle Feature Engineering Class: https://www.kaggle.com/learn/feature-engineering

Taking ideas from that class the following additional features were created. 

The 'Features' feature was created by going through the feature list and finding features that could have a value of 0 and combining the scores of each feature that didn't have a 0. 

The 'MedNhbdArea' feature was derived from two of the most impactful features from doing a Mutual Information score analysis and grouping them to show the average GrLivArea for each Neighborhood.  

In [22]:
Features = ['TotalBsmtSF', 'CentralAir', '2ndFlrSF', 'Fireplaces', 'GarageArea', 'WoodDeckSF', 'PoolArea', 'MiscVal']
train_test['Features'] = train_test[Features].gt(0).sum(axis=1)

train_test['MedNhbdArea'] = (train_test.groupby('Neighborhood')['GrLivArea'].transform('median')) 

The 'Cluster' feature was created using K-Means. The space columns were some of the highest scoring Mutual Information features. They are then scaled, and a K-means analysis was run on them. 

In [23]:
space = ['LotArea', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'GarageArea']

train_test_scaled = train_test.loc[:, space]
train_test_scaled = (train_test_scaled - train_test_scaled.mean(axis=0)) / train_test_scaled.std(axis=0)

kmeans = KMeans(n_clusters=10, random_state=0)
train_test['Cluster'] = kmeans.fit_predict(train_test_scaled)

Next, each of the 10 K-means clusters were used to create 10 new Centroid features based the cluster distances. 

In [24]:
train_test_cd = kmeans.fit_transform(train_test_scaled)

train_test_cd = pd.DataFrame(train_test_cd, columns=[f"Centroid_{i}" for i in range(train_test_cd.shape[1])])
train_test = train_test.join(train_test_cd)
train_test_cd.head()

,Centroid_0,Centroid_1,Centroid_2,Centroid_3,Centroid_4,Centroid_5,Centroid_6,Centroid_7,Centroid_8,Centroid_9
0,2.805477,4.056948,2.918934,1.371359,3.092016,3.354219,2.737796,4.613459,2.630429,0.727621
1,4.203703,2.481990,1.163126,2.948924,0.781183,2.571052,3.407897,3.729459,1.440709,2.752441
2,3.536584,3.683415,3.077031,1.904208,3.041460,3.746463,2.217210,4.016120,2.690726,0.576227
3,3.320031,3.894243,3.161313,1.941166,3.178007,3.675633,2.500895,4.346580,2.627652,0.844579
4,4.968461,3.005393,4.072529,3.502604,3.319298,5.060120,1.077796,3.402015,3.517645,1.913878


Principal Component Analysis was used with eight of the top Mutual Information scoring features. A number of the components showed potential inverse relationships between two features, hence the new features below were created. 

In [25]:
train_test['Neighbor_or_YearBlt'] = train_test['Neighborhood'] * train_test['YearBuilt']
train_test['GrLivArea_or_1stFlrSF'] = train_test['GrLivArea'] * train_test['1stFlrSF']
train_test['YearBlt_or_BsmtQual'] = train_test['YearBuilt'] * train_test['BsmtQual']
train_test['BrLivArea_or_GarageArea'] = train_test['GrLivArea'] * train_test['GarageArea']

To create a column with the log of the SalePrice to match the evaluation metric in the contest. 

In [26]:
train_test['LogSalePrice'] = train_test['SalePrice'].apply(np.log)

#  Setting Up and Running the Models

To separate the train_test dataset back into the train and test datasets and identify the independent and dependent columns. Because the dataset is so small, the cross fold validation process seemed to have much less overfitting than creating the separate validation and training sets. 

In [27]:
train = train_test[train_test['SalePrice'].notnull()].copy()
test = train_test[train_test['SalePrice'].isnull()].drop(['SalePrice','LogSalePrice'],axis=1)
X = train.drop(['SalePrice','LogSalePrice'],axis=1)
y = train.LogSalePrice

In [28]:
X,y = shuffle(X,y, random_state=42)
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

To set up the cross validation folds.

In [29]:
kf = KFold(n_splits=12, random_state=42, shuffle=True)

In a separate workbook, Optuna (https://optuna.org/) was used to find the optimal parameters for a wide selection of regression models. It was interesting that the results using Optuna on this dataset were not significantly better than using the default parameters for most of the models. 

In [30]:
cat_model = CatBoostRegressor(colsample_bylevel= 0.08309602563537534, learning_rate= 0.08286145675756133, depth= 4, 
                              l2_leaf_reg= 14.555249413444315, subsample= 0.9097411584295835, 
                              bagging_temperature= 3.177590955252409, model_size_reg= 0.3808343022980778, 
                              boosting_type= 'Plain', verbose=False, random_state=42)

xgb_model = xgb.XGBRegressor(reg_lambda= 5.096430599223015, alpha= 1.3223698329753615, 
                             colsample_bytree= 0.10366646849522436, subsample= 0.9705075073357825, 
                             learning_rate= 0.04681670801808327, n_estimators= 2340, max_depth= 3, 
                             min_child_weight= 2, num_parallel_tree= 1,
                             objective= 'reg:squarederror', random_state=42)

lr_model = LinearRegression()

br_model = BayesianRidge(alpha_init= 1.4228919418021304, lambda_init= 0.03899454131279402, compute_score=True)

lightgbm_model = LGBMRegressor(objective='regression', 
                       num_leaves=9,
                       learning_rate=0.0019200325083996161, 
                       n_estimators=7000,
                       max_bin=200, 
                       subsample= 0.40495582495561083, 
                       bagging_fraction=0.4240987934393466,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.1309662269268009,
                       feature_fraction_seed=8,
                       min_sum_hessian_in_leaf = 1,
                       verbose=-1,
                       random_state=42)

svr_model = make_pipeline(RobustScaler(), SVR(C=29.994348636600485, epsilon= 0.019482341995527, 
                                              gamma=0.0009992835981754227))

dtr_model = DecisionTreeRegressor(random_state=42)

rfr_model = RandomForestRegressor(n_estimators=1200,
                          max_depth=10,
                          min_weight_fraction_leaf=0,
                          min_samples_split=5,
                          min_samples_leaf=1,
                          max_leaf_nodes=90,
                          max_features=None,
                          oob_score=True,
                          random_state=42)

gbr_model = GradientBoostingRegressor(n_estimators=2200,
                                learning_rate=0.05005099614548874,
                                max_depth=2,
                                max_features='sqrt',
                                min_samples_leaf=3,
                                min_samples_split=24,
                                loss='huber',
                                random_state=42)


To define the scoring metric. 

In [31]:
def rmse_cv(model,X,y):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kf))
    return rmse

To define the model names and models and then create a loop to run each model and print out their scores. Unfortunately, there doesn't seem to be an easy way to suppress the output statements from CatBoost nor the warning statements from LGBMRegressor. So there will be a lot of unnecessary verbiage below. 

In [32]:
model_names = ['CatBoostRegressor','XGBRegressor','LinearRegression','BayesianRidge','LGBMRegressor',
               'SVR','DecisionTreeRegressor','RandomForestRegressor','GradientBoostingRegressor']

In [33]:
models = [cat_model,xgb_model,lr_model,br_model,lightgbm_model,svr_model,dtr_model,rfr_model,gbr_model]

In [34]:
score_rmse = []
for i in models:
    rmse = np.sqrt(-cross_val_score(i, X, y, scoring="neg_mean_squared_error", cv=kf))
    ave_rmse = np.mean(rmse)
    score_rmse.append(ave_rmse)

[LightGBM] [Warning] feature_fraction is set=0.1309662269268009, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1309662269268009
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] bagging_fraction is set=0.4240987934393466, subsample=0.40495582495561083 will be ignored. Current value: bagging_fraction=0.4240987934393466
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.1309662269268009, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1309662269268009
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] bagging_fraction is set=0.4240987934393466, subsample=0.40495582495561083 will be ignored. Current value: bagging_fraction=0.424

In [35]:
model_scores = list(zip(model_names, score_rmse))

In [36]:
model_scores

[('CatBoostRegressor', 0.11669236576542252),
 ('XGBRegressor', 0.11713185871905746),
 ('LinearRegression', 0.13032006816252495),
 ('BayesianRidge', 0.13086931854309383),
 ('LGBMRegressor', 0.11772110599492115),
 ('SVR', 0.13006520771776484),
 ('DecisionTreeRegressor', 0.20708369204073454),
 ('RandomForestRegressor', 0.13944195125990577),
 ('GradientBoostingRegressor', 0.11582818229418641)]

To stack up all the models above except CatBoost, optimized using linear regressor. For some reason CatBoost wasn't able to run correctly using the StackingCVRegressor model. The decision tree regressor model was also removed since it had a significantly higher score than the other models. 

In [37]:
stackCVreg = StackingCVRegressor(regressors=(xgb_model,lr_model,br_model,lightgbm_model,svr_model,
                                            rfr_model,gbr_model),
                                meta_regressor=lr_model,
                                use_features_in_secondary=True,
                                random_state=42)

In [38]:
print(rmse_cv(stackCVreg,np.array(X),np.array(y)).mean())

[LightGBM] [Warning] feature_fraction is set=0.1309662269268009, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1309662269268009
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] bagging_fraction is set=0.4240987934393466, subsample=0.40495582495561083 will be ignored. Current value: bagging_fraction=0.4240987934393466
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.1309662269268009, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1309662269268009
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] bagging_fraction is set=0.4240987934393466, subsample=0.40495582495561083 will be ignored. Current value: bagging_fraction=0.424

[LightGBM] [Warning] feature_fraction is set=0.1309662269268009, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1309662269268009
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] bagging_fraction is set=0.4240987934393466, subsample=0.40495582495561083 will be ignored. Current value: bagging_fraction=0.4240987934393466
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.1309662269268009, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1309662269268009
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] bagging_fraction is set=0.4240987934393466, subsample=0.40495582495561083 will be ignored. Current value: bagging_fraction=0.424

[LightGBM] [Warning] feature_fraction is set=0.1309662269268009, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1309662269268009
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] bagging_fraction is set=0.4240987934393466, subsample=0.40495582495561083 will be ignored. Current value: bagging_fraction=0.4240987934393466
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.1309662269268009, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1309662269268009
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] bagging_fraction is set=0.4240987934393466, subsample=0.40495582495561083 will be ignored. Current value: bagging_fraction=0.424

[LightGBM] [Warning] feature_fraction is set=0.1309662269268009, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1309662269268009
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] bagging_fraction is set=0.4240987934393466, subsample=0.40495582495561083 will be ignored. Current value: bagging_fraction=0.4240987934393466
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.1309662269268009, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1309662269268009
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] bagging_fraction is set=0.4240987934393466, subsample=0.40495582495561083 will be ignored. Current value: bagging_fraction=0.424

[LightGBM] [Warning] feature_fraction is set=0.1309662269268009, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1309662269268009
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] bagging_fraction is set=0.4240987934393466, subsample=0.40495582495561083 will be ignored. Current value: bagging_fraction=0.4240987934393466
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.1309662269268009, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1309662269268009
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] bagging_fraction is set=0.4240987934393466, subsample=0.40495582495561083 will be ignored. Current value: bagging_fraction=0.424

To fit all the models in preparation for creating an overall blended model. 

In [39]:
cat_model_all_data = cat_model.fit(X,y)
xgb_model_all_data = xgb_model.fit(X,y)
lr_model_all_data = lr_model.fit(X,y)
br_model_all_data = br_model.fit(X,y)
lightgbm_model_all_data = lightgbm_model.fit(X,y)
svr_model_all_data = svr_model.fit(X,y)
rfr_model_all_data = rfr_model.fit(X,y)
gbr_model_all_data = gbr_model.fit(X,y)

[LightGBM] [Warning] feature_fraction is set=0.1309662269268009, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1309662269268009
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] bagging_fraction is set=0.4240987934393466, subsample=0.40495582495561083 will be ignored. Current value: bagging_fraction=0.4240987934393466
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


In [40]:
stackCVreg_model = stackCVreg.fit(np.array(X), np.array(y))

[LightGBM] [Warning] feature_fraction is set=0.1309662269268009, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1309662269268009
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] bagging_fraction is set=0.4240987934393466, subsample=0.40495582495561083 will be ignored. Current value: bagging_fraction=0.4240987934393466
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.1309662269268009, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1309662269268009
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] bagging_fraction is set=0.4240987934393466, subsample=0.40495582495561083 will be ignored. Current value: bagging_fraction=0.424

Many of the Kaggle submissions used this type of blending approach. Using 35% for the StackingCVRegressor produced the lowest Kaggle score. Both 30% and 40% produced higher scores. This blended model had noticeable lower scores than the submissions from any of the other single models. 

In [41]:
def blended_predictions(X):
    random_state=42
    return ((.12 * svr_model_all_data.predict(X)) + \
            (.12 * cat_model_all_data.predict(X)) + \
            (.12 * gbr_model_all_data.predict(X)) + \
            (.12 * xgb_model_all_data.predict(X)) + \
            (.12 * lightgbm_model_all_data.predict(X)) + \
            (.05 * rfr_model_all_data.predict(X)) + \
            (0.35 * stackCVreg_model.predict(np.array(X))))

In [42]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [43]:
blended_score = rmsle(y, blended_predictions(X))
print('RMLSE score on train data:')
print(blended_score)

RMLSE score on train data:
0.0720280357516632


To create the submission dataset and to reset the dependent variable of SalePrice back from the earlier conversion to the log of those values. 

In [44]:
submit = test[['Id']]
submit = submit.reset_index(drop=True)

In [45]:
submit_predict = blended_predictions(test)
submit_predict = np.exp(submit_predict)

In [46]:
submit['SalePrice'] = submit_predict

In [47]:
submit.to_csv('submit_features_blended.csv', index=False)